In [ ]:
using Plots
import Plots: px
theme(:default)
ENV["LINES"] = 20

In [ ]:
include("parser.jl")
include("models.jl")
gpu()
download(SIGDataSet)

In [ ]:
data  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-train-high"));
vocab = Vocabulary(data)
edata = encode(data,vocab);
test  = map(parseDataLine, eachline("./data/Sigmorphon/task1/all/spanish-test"));
tdata = encode(test,vocab);
dictionary = [parseDataLine(line) for line in  eachline("./data/unimorph/spa/spa") if line != ""];

In [ ]:
vocab.chars.toElement;

In [ ]:
vocab.specialIndices

In [150]:
trainsfs    = unique(map(x->join(x.surface),data));
testsfs     = unique(map(x->join(x.surface),test)); 
dictsfs     = unique([map(x->join(x.surface),dictionary); trainsfs; testsfs]);
unseensfs   = [x for x in dictsfs if x ∉ trainsfs]

314523-element Array{String,1}:
 "ababilladas"  
 "ababillada"   
 "ababillados"  
 "ababillado"   
 "ababillándose"
 "ababillaos"   
 "ababillarse"  
 "ababíllate"   
 ⋮              
 "zuzase"       
 "zuzasteis"    
 "zuzaste"      
 "zuzas"        
 "zuza"         
 "zuzo"         
 "zuzó"         

In [ ]:
morph = EncAttentiveVAE(length(vocab.chars); H=512, E=16, Z=16)
train_ae!(morph, data, vocab; optim=Adam(), epoch=50)

In [ ]:
morphv =  EncAttentiveVAE(length(vocab.chars); H=512, E=16, Z=16)
copytoparams(morphv.encoder, morph.encoder)
copytoparams(morphv.Wμ, morph.Wμ)
copytoparams(morphv.Wσa, morph.Wσa)
copytoparams(morphv.Wμa, morph.Wμa)
copytoparams(morphv.Wσ, morph.Wσ)
copytoparams(morphv.Weaμ, morph.Weaμ)
copytoparams(morphv.Weaσ, morph.Weaσ)
morphv.decoder.embedding = morphv.encoder.embedding
train_vae!(morphv, data, vocab; optim=Adam(), epoch=45, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=3)

In [ ]:
samples = sample(morphv, vocab, edata; N=10000);

In [ ]:
unique(samples[findall([s ∈ trainsfs for s in samples])])

In [ ]:
length(samples[findall([s ∈ trainsfs for s in samples])])

In [ ]:
unique(samples[findall([s ∈ testsfs for s in samples])])

In [ ]:
length(samples[findall([s ∈ testsfs for s in samples])])

In [ ]:
unique(length(samples[findall([s ∈ dictsfs for s in samples])]))

In [ ]:
sampleinter(morphv, vocab, data; N=10)

In [ ]:
au, sigma, mu = calc_au(morphv,tdata; delta=0.01)

In [ ]:
mi = calc_mi(morphv,tdata)

In [ ]:
calc_ppl(morphv,tdata)

In [ ]:
atts = attentions(morph,edata, vocab);

In [ ]:
function getsingle(sfs,exs,perms, αs,i) 
    join(vocab.chars[sfs[i]]), map(e->join(vocab.chars[e]),exs[i][perms[i]]), map(x->x[:,perms[i],i],αs)
end

In [ ]:
i = rand(1:length(atts))
s1,e1,α1 =  getsingle(atts[i]...,rand(1:16))
ys = 1:512
h1 = heatmap(e1, ys, α1.αu; size=(1200,800), xtickfont = font(10, "Halvetica"), title= s1 * ", mu", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);
h2 = heatmap(e1, ys, α1.ασ ;size=(1200,800), xtickfont = font(10, "Halvetica"), title=s1 * ", sigma", xrotation = 45, titlefontsize=10, left_margin=100px, bottom_margin=100px, right_margin=100px);

In [ ]:
println(s1);println(e1)

In [ ]:
h1

# To-Dos
1. Add unrelated surface forms to the examplars too see whether the attention mappings are meaningful
2. Comparing with Normal Vae
3. Different Sampling Function on LM ($top_k$, temperature)
4. Attention in Decoder
5. Attention to the hidden state sequence, not just to the final state.
6. Metric for testing generation quality?

In [235]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [236]:
morph3=nothing; morphv=nothing; morph=nothing; morph2=nothing; KnetLayers.gc();

In [237]:
morph2 = VAE(length(vocab.chars); H=512, E=16, Z=16, concatz=true)
train_ae!(morph2, data, vocab; optim=Adam(), B=16, epoch=20);

(loss = 24.226039373779297,)
(loss = 17.94939325866699,)
(loss = 14.63141618347168,)
(loss = 11.688678450012207,)
(loss = 9.555169277191162,)
(loss = 7.757795697784424,)
(loss = 6.42142120513916,)
(loss = 5.461640008163452,)
(loss = 4.630353671264649,)
(loss = 3.879816455459595,)
(loss = 3.2313405588150026,)
(loss = 2.7009236671447754,)
(loss = 2.2843154739379883,)
(loss = 1.964221185016632,)
(loss = 1.8177687578201294,)
(loss = 1.5614021615982057,)
(loss = 1.3386266157627105,)
(loss = 1.3330284670352937,)
(loss = 1.1747992154598237,)
(loss = 1.0711584807395935,)


In [222]:
length(vocab.chars)

38

In [210]:
morph3 = VAE(length(vocab.chars); H=512, E=16, Z=16)
copytoparams(morph3.encoder, morph2.encoder)
copytoparams(morph3.Wμ, morph2.Wμ)
copytoparams(morph3.Wσ, morph2.Wσ)
copytoparams(morph3.dec_embed, morph3.encoder.embedding)
train_vae!(morph3, data, vocab; B=16, optim=Adam(lr=0.002), epoch=40, kl_weight=0.0f0, kl_rate = 0.1f0, fb_rate=4)

(kl_weight = 0.1f0, fbr = 4, loss = 21.441761749267577)
(kl_weight = 0.2f0, fbr = 4, loss = 20.253545791625978)
(kl_weight = 0.3f0, fbr = 4, loss = 19.015092526245116)


InterruptException: InterruptException:

In [ ]:
samples = sample(morph3, vocab, edata; N=10000, useprior=true)

In [212]:
samples[findall([s ∈ trainsfs for s in samples])]

InterruptException: InterruptException:

In [213]:
samples[findall([s ∈ testsfs for s in samples])]

6-element Array{String,1}:
 "okupara"      
 "no enamore"   
 "situarías"    
 "vibrase"      
 "no deleguemos"
 "refundamos"   

In [214]:
samples[findall([s ∈ dictsfs for s in samples])]

InterruptException: InterruptException:

In [215]:
samples[findall([s ∈ unseensfs for s in samples])] 

InterruptException: InterruptException:

In [216]:
inters = sampleinter(morph3, vocab, data; N=10)

12-element Array{String,1}:
 "contrastemos"
 "contrastemos"
 "contrasteis" 
 "contrasteis" 
 "contrasteis" 
 "contrasteis" 
 "sobrecaríais"
 "respararíais"
 "recontaríais"
 "recontaríais"
 "recontaríais"
 "recontaríais"

In [217]:
findall([s ∈ dictsfs for s in inters])

6-element Array{Int64,1}:
  1
  2
  9
 10
 11
 12

In [218]:
au, sigma, mu = calc_au(morph3, tdata; delta=0.01)

(16, Float32[0.394202; 0.30743402; … ; 0.62694967; 0.25517163], K32(16,1)[0.03776209⋯])

In [219]:
mi =  calc_mi(morph3,tdata)

3.1721899108886724

In [ ]:
calc_ppl(morph3, edata; nsample=500, B=16)

In [138]:
include("models.jl")
include("parser.jl")

nsample_packed_sequence (generic function with 1 method)

In [ ]:
model = LSTM_LM(length(vocab.chars); H=768, E=16)

In [ ]:
train_rnnlm!(model, data, vocab; epoch=40, optim=Adam(), B=16)

In [ ]:
 calc_ppllm(model, tdata)

In [ ]:
samples = sample(model, vocab; N=10000)

In [ ]:
samples[findall([s ∈ trainsfs for s in samples])]

In [ ]:
samples[findall([s ∈ testsfs for s in samples])]

In [ ]:
samples[findall([s ∈ dictsfs for s in samples])]

In [ ]:
 calc_ppllm(model, edata)

In [202]:
?@eval

```
@eval [mod,] ex
```

Evaluate an expression with values interpolated into it using `eval`. If two arguments are provided, the first is the module to evaluate in.


In [9]:
include("main.jl")

main (generic function with 17 methods)

In [ ]:
exresult = main()

(loss = 24.53541962890625,)
(loss = 18.032949095153807,)
(loss = 15.124377491760255,)
(loss = 11.992093420410157,)
(loss = 9.456983060455322,)
(loss = 7.691069605255127,)
(loss = 6.244724300384521,)
(loss = 5.008035516357422,)
(loss = 4.108970711135864,)
(loss = 3.245434301185608,)
(loss = 2.7649883628845213,)
(loss = 2.3119849225997924,)
(loss = 2.074958394432068,)
(loss = 1.8084550073623658,)
(loss = 1.5498729182720183,)
(